In [1]:
import os 
import numpy as np 
import h5py 
import json
import shutil
import time 
import torch 
import glob 
import scipy.io as sio

from tqdm import tqdm
from datetime import datetime
from utils import *

# torch computation device to calculate jacobian via autograd
# device = torch.device('cuda:0')
device = torch.device('cpu')

/home/nikolai/dev/constrained_inversion_mrm/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Selective Inversion: Peak $B_1$ and Energy Reduction

In [2]:
opt_pulse_directory = 'data/sel_20250313_073618'
opt_pulse_param_file = os.path.join(opt_pulse_directory, 'params_selective.json')

pulse_inds = [0, 1, 2]

with open(opt_pulse_param_file,'r') as fid: par = json.load(fid)
gamma_bar = par[0]['gamma_bar']
dt = par[0]['raster_time']

freq_eval = np.linspace(-4000.0, 4000.0, 512, dtype=np.float64)
b1_eval = np.linspace(0.0, 2.0, 512, dtype=np.float64)

pulses = []
tv = []
dur = []
mz = []
mz_eval = []
show_optimized_region = []
x_corner = []
y_corner = []
width = []
height = []
legend_locs = []
titles = []

for p in range(len(pulse_inds)):

    opt_pulse_file = os.path.join(opt_pulse_directory, 'pulse_%03d.h5'%(pulse_inds[p]))

    # load optimized pulse and reference HS pulse
    with h5py.File(opt_pulse_file, 'r') as F:
        rf_opt = np.array(F['rf_opt'], dtype=np.complex128) * 1e6 / gamma_bar # uT
        rf_hs = np.array(F['rf_hs'], dtype=np.complex128) * 1e6 / gamma_bar   # uT
        mz_hs = np.array(F['mz_hs'], dtype=np.float64)
        mz_opt = np.array(F['mz_opt'], dtype=np.float64)
        b1_design = np.array(F['b1_scales'], dtype=np.float64) 
        freq_design = np.array(F['freq_arr'], dtype=np.float64)
    dur_opt = rf_opt.size * dt 
    dur_hs = rf_hs.size * dt 

    # simulate magnetization 
    mz_opt_eval = simulate_mz(rf_opt*1e-6*gamma_bar, freq_eval, b1_eval, dt, device=device, return_numpy=True)
    mz_hs_eval = simulate_mz(rf_hs*1e-6*gamma_bar, freq_eval, b1_eval, dt, device=device, return_numpy=True)

    # HS pulse (only for p = 0 as this will be the same for all pulses)
    if p == 0:
        pulses.append(rf_hs)
        tv.append(1e3*np.arange(0.0, dur_hs, dt, dtype=np.float64))
        dur.append(dur_hs)
        mz.append(mz_hs) 
        mz_eval.append(mz_hs_eval) 
        show_optimized_region.append(0)
        x_corner.append(0)
        y_corner.append(0)
        width.append(0)
        height.append(0)
        legend_locs.append('upper right')
        titles.append('$\mathrm{HS\ Pulse\ (Initial\ Guess)}$')

    pulses.append(rf_opt)
    tv.append(1e3*np.arange(0.0, dur_opt, dt, dtype=np.float64))
    dur.append(dur_opt)
    mz.append(mz_opt) 
    mz_eval.append(mz_opt_eval) 
    show_optimized_region.append(1)
    x_corner.append(freq_design.min())
    y_corner.append(b1_design.min())
    width.append(freq_design.max() - freq_design.min())
    height.append(b1_design.max() - b1_design.min())
    legend_locs.append('upper right')
    titles.append('$\mathrm{%d\%%\ HS\ Power}$'%(100*par[pulse_inds[p]]['max_energy_fraction_of_hs_reference']))

num_pulses = len(pulses)

eval_file = os.path.join(opt_pulse_directory, 'eval_data.h5')
with h5py.File(eval_file,'w') as F:
    F.create_dataset('num_pulses', data=num_pulses)
    F.create_dataset('b1_design', data=b1_design)
    F.create_dataset('freq_design', data=freq_design)
    F.create_dataset('b1_eval', data=b1_eval)
    F.create_dataset('freq_eval', data=freq_eval)
    for p in range(len(pulses)):
        F.create_dataset('pulse_%i'%(p), data=pulses[p]) 
        F.create_dataset('tv_%i'%(p), data=tv[p])
        F.create_dataset('dur_%i'%(p), data=dur[p]*1e3)
        F.create_dataset('mz_%i'%(p), data=mz[p])
        F.create_dataset('mz_eval_%i'%(p), data=mz_eval[p])
        F.create_dataset('show_optimized_region_%i'%(p), data=show_optimized_region[p]) 
        F.create_dataset('x_corner_%i'%(p), data=x_corner[p]) 
        F.create_dataset('y_corner_%i'%(p), data=y_corner[p]) 
        F.create_dataset('width_%i'%(p), data=width[p]) 
        F.create_dataset('height_%i'%(p), data=height[p]) 
        F.create_dataset('title_%d'%(p), data=titles[p], dtype=h5py.string_dtype(encoding='utf-8'))
        F.create_dataset('legend_loc_%d'%(p), data=legend_locs[p], dtype=h5py.string_dtype(encoding='utf-8'))


# Non-Selective Inversion w/ Comparison to Graf et. al. 

In [7]:
opt_pulse_directory = 'data/nonsel_20250313_163627'
opt_pulse_param_file = os.path.join(opt_pulse_directory, 'params_nonsel.json')

freq_eval = np.linspace(-1000.0, 1000.0, 512, dtype=np.float64)
b1_eval = np.linspace(0.0, 2.0, 512, dtype=np.float64)
b1_high_res = np.linspace(0.8, 1.2, 256, dtype=np.float64)

freq_0_150 = np.array([0.0, 150.0], dtype=np.float64)
with open(opt_pulse_param_file,'r') as fid: par = json.load(fid)
gamma_bar = par[0]['gamma_bar']
dt = par[0]['raster_time']

# get reference HS pulse
opt_pulse_file = os.path.join(opt_pulse_directory, 'pulse_000.h5')
with h5py.File(opt_pulse_file, 'r') as F:
    rf_hs = np.array(F['rf_hs'], dtype=np.complex128) * 1e6 / gamma_bar   # uT
    mz_hs = np.array(F['mz_hs'], dtype=np.float64)
    b1_design = np.array(F['b1_scales'], dtype=np.float64) 
    freq_design = np.array(F['freq_arr'], dtype=np.float64)
dur_hs = rf_hs.size * dt 
mz_hs_eval = simulate_mz(rf_hs*1e-6*gamma_bar, freq_eval, b1_eval, dt, device=device, return_numpy=True)
tv_hs = 1e3 * np.arange(0, dur_hs, dt, dtype=np.float64)
mz_hs_calc = simulate_mz(rf_hs*1e-6*gamma_bar, freq_0_150, b1_high_res, dt, device=device, return_numpy=True)
print('HS Pulse Median Percent Inversion at 0 Hz: %f'%(100*np.median((1-mz_hs_calc[0,:])/2)))
print('HS Pulse Mean Percent Inversion at 0 Hz: %f\n'%(100*np.mean((1-mz_hs_calc[0,:])/2)))
print('HS Pulse Median Percent Inversion at 150 Hz: %f'%(100*np.median((1-mz_hs_calc[1,:])/2)))
print('HS Pulse Mean Percent Inversion at 150 Hz: %f\n'%(100*np.mean((1-mz_hs_calc[1,:])/2)))

# load the pulse provided by Graf et al
graf_pulse_file = 'data/pulse_graf_et_al.mat'
df = sio.loadmat(graf_pulse_file)
u = np.squeeze(df['u']).astype(np.float64)
v = np.squeeze(df['v']).astype(np.float64)
rf_graf = (u + 1j*v) * 1e3 # uT
dt_graf = 0.01*1e-3
dur_graf = rf_graf.size * dt_graf
mz_graf = simulate_mz(rf_graf*1e-6*gamma_bar, freq_design, b1_design, dt_graf, device=device, return_numpy=True)
mz_graf_eval = simulate_mz(rf_graf*1e-6*gamma_bar, freq_eval, b1_eval, dt_graf, device=device, return_numpy=True)
tv_graf = 1e3 * np.arange(0, dur_graf, dt_graf, dtype=np.float64)
mz_graf_calc = simulate_mz(rf_graf*1e-6*gamma_bar, freq_0_150, b1_high_res, dt_graf, device=device, return_numpy=True)
print('Graf Pulse Median Percent Inversion at 0 Hz: %f'%(100*np.median((1-mz_graf_calc[0,:])/2)))
print('Graf Pulse Mean Percent Inversion at 0 Hz: %f\n'%(100*np.mean((1-mz_graf_calc[0,:])/2)))
print('Graf Pulse Median Percent Inversion at 150 Hz: %f'%(100*np.median((1-mz_graf_calc[1,:])/2)))
print('Graf Pulse Mean Percent Inversion at 150 Hz: %f\n'%(100*np.mean((1-mz_graf_calc[1,:])/2)))

pulses = [rf_hs, rf_graf]
tv = [tv_hs, tv_graf]
dur = [dur_hs, dur_graf] # scalar
mz = [mz_hs, mz_graf]
mz_eval = [mz_hs_eval, mz_graf_eval]
show_optimized_region = [0, 1]
x_corner = [0, 0.0]
y_corner = [0, 0.7] # value of 0.7 hard-coded from Graf et. al.
width = [0, 0.0]
height = [0, 1.3 - 0.7]
legend_locs = ['upper right', 'lower right']
titles = ['$\mathrm{HS\ Reference}$', '$\mathrm{Graf\ et.\ al.\ (optim\ }B_1^+\mathrm{)}$']

opt_files = glob.glob(os.path.join(opt_pulse_directory, 'pulse_*.h5'))
for idx, opt_pulse_file in enumerate(opt_files):
    with h5py.File(opt_pulse_file, 'r') as F:
        rf_opt = np.array(F['rf_opt'], dtype=np.complex128) * 1e6 / gamma_bar # uT
        mz_opt = np.array(F['mz_opt'], dtype=np.float64)
    dur_opt = rf_opt.size * dt 
    mz_opt_eval = simulate_mz(rf_opt*1e-6*gamma_bar, freq_eval, b1_eval, dt, device=device, return_numpy=True)
    tv_opt = 1e3 * np.arange(0, dur_opt, dt, dtype=np.float64)
    mz_opt_calc = simulate_mz(rf_opt*1e-6*gamma_bar, freq_0_150, b1_high_res, dt, device=device, return_numpy=True)
    print('Opt Pulse %d Median Percent Inversion at 0 Hz: %f'%(idx, 100*np.median((1-mz_opt_calc[0,:])/2)))
    print('Opt Pulse %d Mean Percent Inversion at 0 Hz: %f\n'%(idx, 100*np.mean((1-mz_opt_calc[0,:])/2)))
    print('Opt Pulse %d Median Percent Inversion at 150 Hz: %f'%(idx, 100*np.median((1-mz_opt_calc[1,:])/2)))
    print('Opt Pulse %d Mean Percent Inversion at 150 Hz: %f\n'%(idx, 100*np.mean((1-mz_opt_calc[1,:])/2)))

    
    pulses.append(rf_opt)
    tv.append(tv_opt)
    dur.append(dur_opt)
    mz.append(mz_opt)
    mz_eval.append(mz_opt_eval)
    show_optimized_region.append(1)
    x_corner.append(freq_design.min())
    y_corner.append(b1_design.min())
    width.append(freq_design.max() - freq_design.min())
    height.append(b1_design.max() - b1_design.min())
    legend_locs.append('center')
    if par[idx]['max_energy_hz_squared_sec'] is None:
        titles.append('$\mathrm{Proposed\ 1}$\n$\mathrm{(Unconstrained\ Power)}$')
    else:
        E_graf = np.sum(np.abs(rf_graf*np.conj(rf_graf))*dt_graf)
        E_opt = np.sum(np.abs(rf_opt*np.conj(rf_opt))*dt)
        pct_red = int(100 * E_opt / E_graf)
        titles.append('$\mathrm{Proposed\ 2}$\n$\mathrm{(%d\%%\ Graf\ et.\ al.\ Power)}$'%(pct_red))


eval_file = os.path.join(opt_pulse_directory, 'eval_data.h5')
with h5py.File(eval_file,'w') as F:
    F.create_dataset('num_pulses', data=num_pulses)
    F.create_dataset('b1_design', data=b1_design)
    F.create_dataset('freq_design', data=freq_design)
    F.create_dataset('b1_eval', data=b1_eval)
    F.create_dataset('freq_eval', data=freq_eval)
    for p in range(len(pulses)):
        F.create_dataset('pulse_%i'%(p), data=pulses[p]) 
        F.create_dataset('tv_%i'%(p), data=tv[p])
        F.create_dataset('dur_%i'%(p), data=dur[p]*1e3)
        F.create_dataset('mz_%i'%(p), data=mz[p])
        F.create_dataset('mz_eval_%i'%(p), data=mz_eval[p])
        F.create_dataset('show_optimized_region_%i'%(p), data=show_optimized_region[p]) 
        F.create_dataset('x_corner_%i'%(p), data=x_corner[p]) 
        F.create_dataset('y_corner_%i'%(p), data=y_corner[p]) 
        F.create_dataset('width_%i'%(p), data=width[p]) 
        F.create_dataset('height_%i'%(p), data=height[p]) 
        F.create_dataset('title_%d'%(p), data=titles[p], dtype=h5py.string_dtype(encoding='utf-8'))
        F.create_dataset('legend_loc_%d'%(p), data=legend_locs[p], dtype=h5py.string_dtype(encoding='utf-8'))



HS Pulse Median Percent Inversion at 0 Hz: 99.943857
HS Pulse Mean Percent Inversion at 0 Hz: 99.936625

HS Pulse Median Percent Inversion at 150 Hz: 99.708707
HS Pulse Mean Percent Inversion at 150 Hz: 99.701079

Graf Pulse Median Percent Inversion at 0 Hz: 99.865082
Graf Pulse Mean Percent Inversion at 0 Hz: 99.849125

Graf Pulse Median Percent Inversion at 150 Hz: 93.645585
Graf Pulse Mean Percent Inversion at 150 Hz: 92.886311

Opt Pulse 0 Median Percent Inversion at 0 Hz: 99.991182
Opt Pulse 0 Mean Percent Inversion at 0 Hz: 99.978928

Opt Pulse 0 Median Percent Inversion at 150 Hz: 99.733861
Opt Pulse 0 Mean Percent Inversion at 150 Hz: 99.701423

Opt Pulse 1 Median Percent Inversion at 0 Hz: 99.957935
Opt Pulse 1 Mean Percent Inversion at 0 Hz: 99.942659

Opt Pulse 1 Median Percent Inversion at 150 Hz: 99.526241
Opt Pulse 1 Mean Percent Inversion at 150 Hz: 99.404761



# Save Non-Selective Pulses for Validation Experiments

In [8]:

opt_pulse_directory = 'data/nonsel_20250313_163627'
opt_pulse_param_file = os.path.join(opt_pulse_directory, 'params_nonsel.json')

with open(opt_pulse_param_file,'r') as fid: par = json.load(fid)
gamma_bar = par[0]['gamma_bar']
dt = par[0]['raster_time']

for n in range(2):
    opt_pulse_file = os.path.join(opt_pulse_directory, 'pulse_%03d.h5'%(n))
    with h5py.File(opt_pulse_file,'r') as F:
        rf = np.array(F['rf_opt'], dtype=np.complex128).astype(np.complex64) # in Hz

    # calculate nominal flip angle in degrees to achieve desired peak B1
    flip_real = np.sum(2*np.pi*np.real(rf)*dt) * 180 / np.pi
    flip_imag = np.sum(2*np.pi*np.imag(rf)*dt) * 180 / np.pi
    flip = np.sqrt(flip_real**2 + flip_imag**2)

    # save the waveform, dwell time, and nominal flip angle to file 
    out_file = os.path.join(opt_pulse_directory, 'pulseq_pulse_%03d.h5'%(n))
    with h5py.File(out_file,'w') as F:
        F.create_dataset('wav', data=rf)
        F.create_dataset('dt', data=dt)
        F.create_dataset('flip', data=flip)

# Compare Simulations with Measurements

In [30]:
data_folder = 'data/measurements_20250319'
data_file = os.path.join(data_folder, 'measured_inversion_factors_with_B0_B1.h5')

# load measured data  
with h5py.File(data_file,'r') as F:
    B0 = np.array(F['B0'], dtype=np.float32).astype(np.float64)
    B1 = np.array(F['B1'], dtype=np.float32).astype(np.float64)
    img = np.array(F['img'], dtype=np.complex64).astype(np.complex128)
    mask = np.array(F['mask'], dtype=np.float32).astype(np.float64)
    inv_scales = np.array(F['inv_scales'], dtype=np.float32).astype(np.float64)

# calculate inversion factor (Equation 4)
IF = 1.0 - 0.5 * np.abs(img[:,:,0,None] + img[:,:,1:]) / np.abs(img[:,:,0,None])

# load the pulse waveform 
pulse_file = 'data/nonsel_20250313_163627/pulse_000.h5'
param_file = 'data/nonsel_20250313_163627/params_nonsel.json'
with open(param_file,'r') as fid: par = json.load(fid)
dt = par[0]['raster_time']
with h5py.File(pulse_file,'r') as F:
    rf = np.array(F['rf_opt'], dtype=np.complex128)

# simulate the inversion factor 
IF_sim = np.zeros_like(IF)
for row in tqdm(range(B0.shape[0])):
    for col in range(B0.shape[1]):
        b0_val = np.array([B0[row,col]], dtype=np.float64)
        b1_val = np.array([B1[row,col]], dtype=np.float64)
        if mask[row,col] > 0.0:
            for scl in range(inv_scales.size):
                mz = simulate_mz(rf * inv_scales[scl], b0_val, b1_val, dt, device=device, return_numpy=True)[0,0]
                IF_sim[row,col,scl] = (1 - mz)/2 

eval_file = os.path.join(data_folder, 'eval_data_meas_vs_sim.h5')
with h5py.File(eval_file,'w') as F:
    F.create_dataset('B0', data=B0)
    F.create_dataset('B1', data=B1)
    F.create_dataset('IF', data=IF)
    F.create_dataset('IF_sim', data=IF_sim)
    F.create_dataset('mask', data=mask)
    F.create_dataset('inv_scales', data=inv_scales)


100%|██████████| 64/64 [02:43<00:00,  2.55s/it]
